Vivekn Sentiment analysis computes Vivek Nayamanan algorithm from a set of positive and negative corpus

We call necessary imports

In [ ]:
#Imports
import time
import sys
import os
sys.path.append('../../')

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


We load SparkSession if not already there

In [ ]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

We load a spark dataset and put it in memory

In [ ]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("file:///" + os.getcwd() + "/../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

We creat the document assemblerr, which will put target text column into Annotation form

In [ ]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

The sentence detector will parse sub sentences in every line

In [ ]:
### Sentence detector
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

The tokenizer will match standard tokens

In [ ]:
### Tokenizer
tokenizer = Tokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

Normalizer will clean out the tokens

In [ ]:
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")

The spell checker will correct normalized tokens, this trains with a dictionary of english words

In [ ]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell") \
            .setDictionary("file:///" + os.getcwd() + "/../../../src/test/resources/spell/words.txt")

#checked = spell_checker.fit(tokenized).transform(tokenized)

We creat the ViveknSentimentApproach and set resources to train it

In [ ]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPruneCorpus(0) \
    .setPositiveSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/negative") \


The finisher will utilize sentiment analysis output

In [ ]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

Dummy training text

In [ ]:
training = spark.sparkContext.parallelize([["hello world"], ["this is some more text"], ["and here another sentence"]]).toDF().toDF("text")
training.show()

We fit and predict over data

In [ ]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    spell_checker,
    sentiment_detector,
    finisher
])

start = time.time()
sentiment_data = pipeline.fit(training).transform(data)
sentiment_data.show()
end = time.time()
print("Time elapsed pipeline process: " + str(end - start))

We can take a sample back into the driver

In [ ]:
for r in sentiment_data.take(5):
    print(r)

We save it to disk and re read it. Either after or before fitting the model

In [ ]:
start = time.time()
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")
end = time.time()
print("Time elapsed in write pipelines: " + str(end - start))

In [ ]:
start = time.time()
p = Pipeline.read().load("./ps")
pm = PipelineModel.read().load("./ms")
end = time.time()
print("Time elapsed in read pipelines: " + str(end - start))

In [ ]:
start = time.time()
pm.transform(data).where("finished_sentiment not like '%negative%'").show()
print(pm.transform(data).count())
end = time.time()
print("Time elapsed in using loaded pipelines: " + str(end - start))